In [ ]:
# %% [markdown]
# # Otimização de Portfólio para Locais de Mineração de Nódulos Polimetálicos
# 
# Este notebook demonstra como usar o LETE e o módulo de otimização para selecionar
# os melhores locais de mineração de nódulos polimetálicos no Atlântico.

# %%
# Importar bibliotecas
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Adicionar o diretório src ao path
sys.path.append(os.path.abspath('../'))

# Importar os módulos personalizados
from src.network.lete import LETECalculator
from src.optimization.portfolio import MiningPortfolioOptimizer

# Configurar visualizações
plt.style.use('seaborn-whitegrid')
sns.set(style="whitegrid", font_scale=1.2)

# %% [markdown]
# ## 1. Preparar Dados de Exemplo
# 
# Vamos usar o mesmo conjunto de dados simulados do notebook anterior, mas nos
# concentrando nos locais de mineração e seus retornos esperados.

# %%
# Gerar dados simulados
np.random.seed(42)

# Período de tempo (meses)
n_periods = 60
dates = pd.date_range(start='2018-01-01', periods=n_periods, freq='M')

# Sites de mineração simulados
sites = ['Rio_Grande_Rise_A', 'Rio_Grande_Rise_B', 'Mid_Atlantic_Ridge', 
         'Bermuda_Rise', 'Azores_Plateau']

# Metais presentes nos nódulos
metals = ['Nickel', 'Copper', 'Cobalt', 'Manganese']

# Gerar preços de metais com tendências e sazonalidades
prices_df = pd.DataFrame(index=dates)
base_prices = {'Nickel': 15000, 'Copper': 8000, 'Cobalt': 30000, 'Manganese': 1700}

for metal in metals:
    # Tendência linear + componente sazonal + componente auto-regressivo + ruído
    trend = np.linspace(0, 0.3, n_periods)
    seasonality = 0.1 * np.sin(np.linspace(0, 6*np.pi, n_periods))
    
    # Componente auto-regressivo
    ar_component = np.zeros(n_periods)
    ar_component[0] = np.random.normal(0, 0.02)
    for i in range(1, n_periods):
        ar_component[i] = 0.8 * ar_component[i-1] + np.random.normal(0, 0.02)
    
    # Preço final
    price = base_prices[metal] * (1 + trend + seasonality + ar_component + np.random.normal(0, 0.05, n_periods))
    prices_df[metal] = price

# Gerar dados dos sites de mineração
site_data = pd.DataFrame(index=dates)

for site in sites:
    # Densidade base varia por site
    base_density = {
        'Rio_Grande_Rise_A': 12.0,
        'Rio_Grande_Rise_B': 10.5,
        'Mid_Atlantic_Ridge': 8.0,
        'Bermuda_Rise': 7.0,
        'Azores_Plateau': 6.5
    }[site]
    
    # A densidade é afetada pelo preço do níquel com defasagem
    nickel_effect = 0.1 * (prices_df['Nickel'].shift(2) - prices_df['Nickel'].mean()) / prices_df['Nickel'].std()
    
    # Também é afetada pelo preço do cobalto com defasagem
    cobalt_effect = 0.05 * (prices_df['Cobalt'].shift(1) - prices_df['Cobalt'].mean()) / prices_df['Cobalt'].std()
    
    # Densidade final (kg/m²)
    density = base_density + nickel_effect + cobalt_effect + np.random.normal(0, 0.2, n_periods)
    site_data[f"{site}_density"] = density.clip(lower=0)

    # Gerar teores dos metais (%)
    for metal in metals:
        # Teor base varia por metal e site
        base_grade = {
            'Nickel': {'Rio_Grande_Rise_A': 1.4, 'Rio_Grande_Rise_B': 1.3, 'Mid_Atlantic_Ridge': 1.1, 
                      'Bermuda_Rise': 1.0, 'Azores_Plateau': 0.9},
            'Copper': {'Rio_Grande_Rise_A': 1.2, 'Rio_Grande_Rise_B': 1.1, 'Mid_Atlantic_Ridge': 1.0, 
                      'Bermuda_Rise': 0.9, 'Azores_Plateau': 0.8},
            'Cobalt': {'Rio_Grande_Rise_A': 0.25, 'Rio_Grande_Rise_B': 0.24, 'Mid_Atlantic_Ridge': 0.22, 
                      'Bermuda_Rise': 0.20, 'Azores_Plateau': 0.18},
            'Manganese': {'Rio_Grande_Rise_A': 25.0, 'Rio_Grande_Rise_B': 24.0, 'Mid_Atlantic_Ridge': 22.0, 
                         'Bermuda_Rise': 21.0, 'Azores_Plateau': 20.0}
        }[metal][site]
        
        # Variação temporal no teor
        grade_variation = np.random.normal(0, 0.05, n_periods)
        
        # Teor final (%)
        grade = base_grade * (1 + grade_variation)
        site_data[f"{site}_{metal}_grade"] = grade.clip(lower=0)

# Gerar custos de extração (USD/ton)
for site in sites:
    # Custo base varia por site
    base_cost = {
        'Rio_Grande_Rise_A': 300,
        'Rio_Grande_Rise_B': 320,
        'Mid_Atlantic_Ridge': 350,
        'Bermuda_Rise': 380,
        'Azores_Plateau': 400
    }[site]
    
    # Custos são afetados pelo preço do petróleo (simulado)
    oil_price = 80 + 20 * np.sin(np.linspace(0, 4*np.pi, n_periods)) + np.random.normal(0, 5, n_periods)
    oil_effect = 0.3 * (oil_price - oil_price.mean()) / oil_price.std()
    
    # Também são afetados pela densidade (maior densidade = menor custo por tonelada)
    density_effect = -0.2 * (site_data[f"{site}_density"] - site_data[f"{site}_density"].mean()) / site_data[f"{site}_density"].std()
    
    # Custo final
    cost = base_cost * (1 + oil_effect + density_effect)
    site_data[f"{site}_cost"] = cost

# %% [markdown]
# ## 2. Calcular Rentabilidade Esperada para Cada Local
# 
# Precisamos calcular a rentabilidade esperada para cada local de mineração,
# com base nos teores de metais, preços e custos.

# %%
# Calcular valor por tonelada para cada site (últimos 12 meses)
recent_period = 12
site_value = pd.DataFrame(index=sites)

for site in sites:
    # Calcular valor bruto por tonelada
    total_value = 0
    
    for metal in metals:
        # Obter preço e teor médios do período recente
        avg_price = prices_df[metal].iloc[-recent_period:].mean()
        avg_grade = site_data[f"{site}_{metal}_grade"].iloc[-recent_period:].mean() / 100  # Converter % para fração
        
        # Contribuição do metal para o valor da tonelada
        metal_value = avg_price * avg_grade
        total_value += metal_value
        
        # Armazenar valor individual do metal
        site_value.loc[site, f"{metal}_value"] = metal_value
    
    # Obter custo médio do período recente
    avg_cost = site_data[f"{site}_cost"].iloc[-recent_period:].mean()
    
    # Calcular valor líquido (lucro por tonelada)
    net_value = total_value - avg_cost
    
    # Armazenar resultados
    site_value.loc[site, 'gross_value'] = total_value
    site_value.loc[site, 'cost'] = avg_cost
    site_value.loc[site, 'net_value'] = net_value
    
    # Calcular ROI (retorno sobre investimento)
    site_value.loc[site, 'roi'] = net_value / avg_cost

# Exibir resultados
site_value

# %%
# Visualizar valor bruto vs. custo
plt.figure(figsize=(12, 6))

# Barras para valor bruto
bars1 = plt.bar(sites, site_value['gross_value'], label='Valor Bruto')

# Barras para custo
bars2 = plt.bar(sites, site_value['cost'], label='Custo', alpha=0.7)

# Adicionar rótulos
plt.xlabel('Local de Mineração')
plt.ylabel('USD/tonelada')
plt.title('Valor Bruto vs. Custo por Local de Mineração')
plt.xticks(rotation=45)
plt.legend()
plt.grid(axis='y', alpha=0.3)

# Ajustar layout
plt.tight_layout()
plt.show()

# %%
# Visualizar composição do valor por metal
plt.figure(figsize=(12, 6))

# Dados para o gráfico de barras empilhadas
metal_values = [site_value[f"{metal}_value"] for metal in metals]

# Criar barras empilhadas
bottom = np.zeros(len(sites))
for i, metal in enumerate(metals):
    plt.bar(sites, site_value[f"{metal}_value"], bottom=bottom, label=metal)
    bottom += site_value[f"{metal}_value"]

# Sobrepor linha de custo
plt.plot(sites, site_value['cost'], 'r--', label='Custo', linewidth=2)

# Adicionar rótulos
plt.xlabel('Local de Mineração')
plt.ylabel('USD/tonelada')
plt.title('Composição do Valor por Metal e Custo')
plt.xticks(rotation=45)
plt.legend()
plt.grid(axis='y', alpha=0.3)

# Ajustar layout
plt.tight_layout()
plt.show()

# %% [markdown]
# ## 3. Calcular a Matriz LETE Entre os Locais de Mineração
# 
# Agora vamos calcular a matriz LETE para quantificar as relações causais entre os diferentes
# locais de mineração.

# %%
# Preparar dados para análise LETE
# Vamos usar os retornos (ROI) mensais de cada site
site_returns = pd.DataFrame(index=dates[1:])

for site in sites:
    # Calcular valor bruto por tonelada para cada mês
    site_value_ts = pd.Series(index=dates, dtype=float)
    
    for t in range(len(dates)):
        # Valor bruto no período t
        total_value = 0
        
        for metal in metals:
            # Preço e teor no período t
            price = prices_df[metal].iloc[t]
            grade = site_data[f"{site}_{metal}_grade"].iloc[t] / 100  # Converter % para fração
            
            # Contribuição do metal para o valor
            metal_value = price * grade
            total_value += metal_value
        
        # Custo no período t
        cost = site_data[f"{site}_cost"].iloc[t]
        
        # Valor líquido (lucro por tonelada)
        net_value = total_value - cost
        
        # ROI
        roi = net_value / cost
        
        # Armazenar ROI
        site_value_ts.iloc[t] = roi
    
    # Calcular retornos (diferença percentual no ROI)
    returns = site_value_ts.pct_change().iloc[1:]
    
    # Armazenar na DataFrame de retornos
    site_returns[site] = returns

# Remover missings
site_returns = site_returns.dropna()

# Calcular matriz LETE
lete_calc = LETECalculator(k=1, l=1, bins=5, shuffle_iterations=50)
lete_matrix = lete_calc.calculate_lete_matrix(site_returns)

# Visualizar matriz LETE
plt.figure(figsize=(10, 8))
sns.heatmap(lete_matrix, annot=True, cmap='viridis', fmt='.3f')
plt.title('Matriz LETE entre Locais de Mineração')
plt.tight_layout()
plt.show()

# %% [markdown]
# ## 4. Otimização de Portfólio Baseada em LETE
# 
# Vamos agora usar o módulo de otimização de portfólio para encontrar a alocação ótima
# de recursos entre os diferentes locais de mineração.

# %%
# Definir retornos esperados (ROI médio recente)
expected_returns = site_value['roi']

# Criar otimizador de portfólio
portfolio_optimizer = MiningPortfolioOptimizer(
    lete_matrix=lete_matrix,
    expected_returns=expected_returns,
    risk_aversion=1.0
)

# Otimizar portfólio
result = portfolio_optimizer.optimize()

# Exibir resultados
print("Pesos ótimos do portfólio:")
print(result['weights'])
print(f"\nRetorno esperado: {result['expected_return']:.2%}")
print(f"Risco (LETE): {result['risk']:.4f}")
print(f"Status: {result['message']}")

# %%
# Visualizar pesos do portfólio
plt.figure(figsize=(10, 6))
result['weights'].plot(kind='bar', color='skyblue')
plt.title('Alocação Ótima de Recursos entre Locais de Mineração')
plt.xlabel('Local de Mineração')
plt.ylabel('Peso no Portfólio')
plt.xticks(rotation=45)
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

# %% [markdown]
# ## 5. Fronteira Eficiente para Diferentes Níveis de Risco
# 
# Vamos calcular a fronteira eficiente para diferentes níveis de aversão ao risco.

# %%
# Calcular e plotar fronteira eficiente
plt.figure(figsize=(12, 8))
ax = portfolio_optimizer.plot_efficient_frontier()

# Marcar o portfólio atual
ax.plot(result['risk'], result['expected_return'], 'ro', markersize=10, label='Portfólio Atual')

# Ajustar layout
plt.legend()
plt.tight_layout()
plt.show()

# %%
# Calcular portfólios ótimos para diferentes níveis de aversão ao risco
risk_preferences = {
    'Conservador': 2.0,
    'Moderado': 1.0,
    'Agressivo': 0.5
}

# Criar DataFrame para comparação
comparison = pd.DataFrame(index=sites)

for preference, risk_aversion in risk_preferences.items():
    # Atualizar aversão ao risco
    portfolio_optimizer.risk_aversion = risk_aversion
    
    # Otimizar portfólio
    result = portfolio_optimizer.optimize()
    
    # Adicionar pesos à comparação
    comparison[preference] = result['weights']
    
    # Imprimir métricas
    print(f"\nPortfólio {preference}:")
    print(f"Retorno esperado: {result['expected_return']:.2%}")
    print(f"Risco (LETE): {result['risk']:.4f}")

# Exibir comparação
comparison

# %%
# Visualizar comparação de portfólios
comparison.plot(kind='bar', figsize=(12, 6))
plt.title('Comparação de Portfólios para Diferentes Perfis de Risco')
plt.xlabel('Local de Mineração')
plt.ylabel('Peso no Portfólio')
plt.xticks(rotation=45)
plt.grid(axis='y', alpha=0.3)
plt.legend(title='Perfil de Risco')
plt.tight_layout()
plt.show()

# %% [markdown]
# ## 6. Visualização da Rede do Portfólio Ótimo
# 
# Vamos visualizar a rede do portfólio ótimo para cada perfil de risco.

# %%
# Visualizar redes dos portfólios
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

for i, (preference, risk_aversion) in enumerate(risk_preferences.items()):
    # Atualizar aversão ao risco
    portfolio_optimizer.risk_aversion = risk_aversion
    
    # Otimizar portfólio
    result = portfolio_optimizer.optimize()
    
    # Plotar rede
    portfolio_optimizer.plot_portfolio_network(result['weights'], threshold=0.01, ax=axes[i])
    
    # Adicionar título
    axes[i].set_title(f'Portfólio {preference}')

plt.tight_layout()
plt.show()

# %% [markdown]
# ## 7. Análise de Sensibilidade
# 
# Vamos analisar como o portfólio ótimo muda com variações nos preços dos metais.

# %%
# Definir cenários de preços
price_scenarios = {
    'Base': {'Nickel': 1.0, 'Copper': 1.0, 'Cobalt': 1.0, 'Manganese': 1.0},
    'Ni+20%': {'Nickel': 1.2, 'Copper': 1.0, 'Cobalt': 1.0, 'Manganese': 1.0},
    'Cu+20%': {'Nickel': 1.0, 'Copper': 1.2, 'Cobalt': 1.0, 'Manganese': 1.0},
    'Co+20%': {'Nickel': 1.0, 'Copper': 1.0, 'Cobalt': 1.2, 'Manganese': 1.0},
    'Mn+20%': {'Nickel': 1.0, 'Copper': 1.0, 'Cobalt': 1.0, 'Manganese': 1.2}
}

# Calcular retornos esperados para cada cenário
scenario_returns = {}

for scenario, price_factors in price_scenarios.items():
    # Calcular novos retornos esperados
    scenario_roi = pd.Series(index=sites, dtype=float)
    
    for site in sites:
        # Calcular valor bruto ajustado
        total_value = 0
        
        for metal in metals:
            # Valor base do metal
            base_value = site_value.loc[site, f"{metal}_value"]
            
            # Aplicar fator de preço
            adjusted_value = base_value * price_factors[metal]
            total_value += adjusted_value
        
        # Custo permanece o mesmo
        cost = site_value.loc[site, 'cost']
        
        # Novo ROI
        roi = (total_value - cost) / cost
        scenario_roi[site] = roi
    
    # Armazenar retornos do cenário
    scenario_returns[scenario] = scenario_roi

# Criar DataFrame para comparação
scenario_comparison = pd.DataFrame(index=sites)

# Aversão ao risco moderada
risk_aversion = 1.0

for scenario, returns in scenario_returns.items():
    # Criar otimizador com novos retornos
    scenario_optimizer = MiningPortfolioOptimizer(
        lete_matrix=lete_matrix,
        expected_returns=returns,
        risk_aversion=risk_aversion
    )
    
    # Otimizar portfólio
    result = scenario_optimizer.optimize()
    
    # Adicionar pesos à comparação
    scenario_comparison[scenario] = result['weights']
    
    # Imprimir métricas
    print(f"\nCenário {scenario}:")
    print(f"Retorno esperado: {result['expected_return']:.2%}")
    print(f"Risco (LETE): {result['risk']:.4f}")

# Exibir comparação
scenario_comparison

# %%
# Visualizar comparação de cenários
scenario_comparison.plot(kind='bar', figsize=(12, 6))
plt.title('Sensibilidade do Portfólio Ótimo a Variações nos Preços dos Metais')
plt.xlabel('Local de Mineração')
plt.ylabel('Peso no Portfólio')
plt.xticks(rotation=45)
plt.grid(axis='y', alpha=0.3)
plt.legend(title='Cenário de Preços')
plt.tight_layout()
plt.show()

# %% [markdown]
# ## 8. Conclusões e Implicações para Viabilidade Econômica
# 
# Neste notebook, demonstramos como a metodologia LETE pode ser aplicada à otimização
# de portfólio para mineração de nódulos polimetálicos no Atlântico. As principais conclusões são:
# 
# 1. A metodologia permite identificar de forma robusta as relações causais entre diferentes
#    locais de mineração, mesmo em condições de alta volatilidade.
# 
# 2. A otimização de portfólio baseada em LETE oferece uma alternativa mais robusta à
#    abordagem tradicional de Markowitz, especialmente para este tipo de investimento de alto risco.
# 
# 3. A análise de sensibilidade mostrou que [discutir principais insights da análise].
# 
# Para a viabilidade econômica da exploração de nódulos polimetálicos no Atlântico, 
# os resultados sugerem que:
# 
# 1. Os locais mais promissores são [discutir locais mais promissores], devido à combinação 
#    favorável de teores de metais, custos operacionais e menor exposição a flutuações de preços.
# 
# 2. A diversificação entre diferentes locais é crucial para mitigar riscos, com alocação
#    ótima variando conforme o perfil de risco do investidor.
# 
# 3. A viabilidade econômica é particularmente sensível a variações nos preços de [discutir metais mais impactantes],
#    o que deve ser considerado no planejamento de longo prazo.
# 
# Próximos passos incluem a aplicação desta metodologia a dados reais de prospecção, 
# incorporação de fatores ambientais e regulatórios, e desenvolvimento de um modelo mais 
# completo que inclua aspectos logísticos e temporais.